# Bayesian Decision Analysis

Allen Downey

[Bayesian Decision Analysis](https://allendowney.github.io/BayesianDecisionAnalysis/)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## The Five Urn Problem

Let's start by solving the Five Urn problem using a pandas `Series` to represent a [Probability Mass Function](https://en.wikipedia.org/wiki/Probability_mass_function) (PMF).

The key idea is that the **index** of the `Series` represents a set of hypotheses and the **values** of the `Series` represent the corresponding probabilities.

You have five urns that contain blue and yellow marbles:

* Urn 0 contains 0% blue marbles.
* Urn 1 contains 25% blue marbles.
* Urn 2 contains 50% blue marbles.
* Urn 3 contains 75% blue marbles.
* Urn 4 contains 100% blue marbles.

You choose an urn, choose a marble, and it's blue.

What is the posterior probability that you chose each urn?

I'll use integers to represent the hypotheses, so `0` represents the hypothesis that we chose from Urn 0.
The prior probabilities are equal for the five hypotheses.

In [2]:
hypos = [0, 1, 2, 3, 4]
prior = pd.Series(1/5, hypos)
prior

If we chose from Urn $i$, the probability of getting a blue marble is $i/4$.
So that's the likelihood of the data.

In [3]:
likelihood = np.array(hypos) / 4
likelihood

Here's the fundamental step of all Bayesian updates, multiplying the prior by the likelihood.

In [4]:
posterior = prior * likelihood
posterior

The sum of these products is the total probability of the data.

In [5]:
prob_data = posterior.sum()
prob_data

The last step is to normalize the posterior probabilities by dividing through by the probability of the data.

In [6]:
posterior /= prob_data
posterior

The results are the same as what we worked through in the slides.

## The Bayesian Bandit Problem

Now let's get to the Bayesian Bandit problem, which is a model for many kinds of A/B testing, including medical trials.

Suppose you have several "one-armed bandit" slot machines, and there's reason to think that they have different probabilities of paying off.

Each time you play a machine, you either win or lose, and you can use the outcome to update your belief about the probability of winning.

Then, to decide which machine to play next, you can use the "Bayesian bandit" strategy, explained below.

First, let's see how to do the update.

## The prior

If we know nothing about the probability of winning, we can start with a uniform prior.
We'll use integers from 0 to 100 to represent hypothetical probabilities of winning (as percentages).

In [9]:
def decorate(title=""):
    """Labels the axes.

    title: string
    """
    plt.xlabel("Probability of winning")
    plt.ylabel("PMF")
    plt.title(title)

## Bayesian Update

The prior represents what we believe about possible values of `x` before we have any data.
Now suppose we play a machine once and win.
What should we believe about `x` now?

We can answer that question by computing the likelihood of the data, a win, for each value of `x`.
If `x` is 50, the probability of winning is 0.5.
If `x` is 75, the probability is 0.75.
In general, we can compute the probabilities by dividing the values of `x` by 100.

In [15]:
posterior.plot()
decorate("Posterior, one win")

Suppose we play the same machine and win again. We can do a second update, using the posterior from the first update as the prior for the second.

In [16]:
posterior2 = posterior * likelihood_win
posterior2 /= posterior2.sum()
posterior2.plot()
decorate("Posterior, two wins")

And suppose we play one more time and lose. Now we need the likelihood of losing for each value of `x`.


And here's the update.

In [18]:
posterior3 = posterior2 * likelihood_loss
posterior3 /= posterior3.sum()
posterior3.plot()
decorate("Posterior, two wins, one loss")

## The update function

The following function takes as parameters a Pandas Series that represents the prior distribution and a string that represents the data: either `W` if we won or `L` if we lost.

In [19]:
def update(pmf, data):
    """Likelihood function for Bayesian bandit

    pmf: Series that maps hypotheses to probabilities
    data: string, either 'W' or 'L'
    """
    if data == "W":
        likelihood = likelihood_win
    else:
        likelihood = likelihood_loss

    pmf *= likelihood
    pmf /= pmf.sum()

It uses the quantities in the index to compute the likelihood of the data, then updates `pmf` by multiplying by the likelihood and dividing through by the probability of the data.

### Exercise

Suppose you play a machine 10 times and win once.  What is the posterior distribution of $x$?

In [21]:
outcomes = "WLLLLLLLLL"

## Multiple bandits

Now suppose we have several bandits and we want to decide which one to play.

For this example, suppose we have 4 machines with these probabilities:

In [23]:
actual_probs = [0.0, 0.1, 0.2, 0.3]

For purposes of the example, we should **assume that we do not know these probabilities**.

The function `play` simulates playing one machine once and returns `W` or `L`.

In [24]:
from random import random


def flip(p):
    """Return True with probability p."""
    return random() < p

In [25]:
from collections import Counter

# count how many times we've played each machine
counter = Counter()


def play(i):
    """Play machine i.

    returns: string 'W' or 'L'
    """
    counter[i] += 1
    p = actual_probs[i]
    if flip(p):
        return "W"
    else:
        return "L"

Here's a test, playing machine 3 ten times:

In [26]:
for i in range(10):
    outcome = play(3)
    print(outcome, end=" ")

Now I'll make four copies of the prior to represent our beliefs about the four machines.

This function displays four distributions in a grid.

In [28]:
def plot(beliefs, **options):
    for i, b in enumerate(beliefs):
        plt.subplot(2, 2, i + 1)
        b.plot(label="Machine %s" % i)
        plt.gca().set_yticklabels([])
        plt.legend()

    plt.tight_layout()

In [29]:
plot(beliefs)

As an example, let's play each machine 10 times, then plot the posterior distributions.  

In [31]:
plot(beliefs)

## Bayesian Bandits

To get more information, we could play each machine 100 times, but while we are gathering data, we are not making good use of it.  The kernel of the Bayesian Bandits algorithm is that it collects and uses data at the same time.  In other words, it balances exploration and exploitation.

To do that, it draws a random value from each distribution and chooses the the machine that generates the largest value.

The following function takes a PMF and chooses a random value from it, using the probabilities as weights.

In [32]:
def pmf_choice(pmf):
    """Draw a random sample from a PMF.

    pmf: Series representing a PMF

    returns: quantity from PMF
    """
    return np.random.choice(a=pmf.index, p=pmf.values)

Here's an example.

In [33]:
pmf_choice(beliefs[0])

The following function uses `pmf_choice` to choose one value from the posterior distribution of each machine and then uses `argmax` to find the index of the machine that chose the highest value.

In [34]:
def choose(beliefs):
    """Use the Bayesian bandit strategy to choose a machine.

    Draws a sample from each distribution.

    returns: index of the machine that yielded the highest value
    """
    ps = [pmf_choice(b) for b in beliefs]
    return np.argmax(ps)

Here's an example.

In [35]:
choose(beliefs)

`choose` has the property that the probability of choosing each machine is equal to its "probability of superiority".

**Exercise 3:** Putting it all together, fill in the following function to choose a machine, play once, and update `beliefs`:

In [36]:
def choose_play_update(beliefs, verbose=False):
    """Chose a machine, play it, and update beliefs.

    beliefs: list of Pmf objects
    verbose: Boolean, whether to print results
    """
    # choose a machine
    machine = ____

    # play it
    outcome = ____

    # update beliefs
    update(____)

    if verbose:
        print(machine, outcome)

Here's an example:

In [38]:
choose_play_update(beliefs, verbose=True)

## Trying it out

Let's start again with a fresh set of machines and an empty `Counter`.

In [39]:
beliefs = [prior.copy() for i in range(4)]
counter = Counter()

If we run the bandit algorithm 100 times, we can see how `beliefs` gets updated:

In [40]:
num_plays = 100

for i in range(num_plays):
    choose_play_update(beliefs)

plot(beliefs)

The estimates are still rough, especially for the lower-probability machines.  But that's a feature, not a bug: the goal is to play the high-probability machines most often.  Making the estimates more precise is a means to that end, but not an end itself.

Let's see how many times each machine got played.  If things go according to plan, the machines with higher probabilities should get played more often.

In [41]:
for machine, count in sorted(counter.items()):
    print(machine, count)

**Exercise 4:**  Go back and run this section again with a different value of `num_play` and see how it does.

## Summary

The algorithm I presented in this notebook is called [Thompson sampling](https://en.wikipedia.org/wiki/Thompson_sampling).  It is an example of a general strategy called [Bayesian decision theory](https://wiki.lesswrong.com/wiki/Bayesian_decision_theory), which is the idea of using a posterior distribution as part of a decision-making process, usually by choosing an action that minimizes the costs we expect on average (or maximizes a benefit).

In my opinion, this strategy is the biggest advantage of Bayesian methods over classical statistics.  When we represent knowledge in the form of probability distributions, Bayes's theorem tells us how to change our beliefs as we get more data, and Bayesian decision theory tells us how to make that knowledge actionable.

Copyright 2022 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)